<a href="https://colab.research.google.com/github/pratham11kaushik/twitter_sentiment_analysis_using_ML.ipynb/blob/main/1ST_twitter_sentiment_analysis_using_ML(Final)_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing the dependencies

In [1]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# printing the stopwords in english
print (stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# data processing

In [4]:
import pandas as pd

# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/tweeter new data.csv', encoding= 'ISO-8859-1')

In [5]:
# CHECKING THE NO. OF rows and columns
twitter_data.shape

(15924, 4)

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [9]:
# naming the  colums and reading the dataset again

column_names = ['id','user platform','target','text']
twitter_data = pd.read_csv('/content/tweeter new data.csv', names=column_names, encoding= 'ISO-8859-1')


# CHECKING THE NO. OF rows and columns
twitter_data.shape

(46430, 4)

In [10]:

# printing the first 5 rows of dataframe
twitter_data.head()

,id,user platform,target,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [11]:
# counting the no. of missing values in the dataset
twitter_data.isnull().sum()

id                 0
user platform      0
target             0
text             433
dtype: int64

In [12]:
twitter_data.fillna(0, inplace=True)

In [13]:
# counting the no. of missing values in the dataset  # mission completed
twitter_data.isnull().sum()

id               0
user platform    0
target           0
text             0
dtype: int64

In [14]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
Positive      13728
Negative      12378
Neutral       11516
Irrelevant     8808
Name: count, dtype: int64

# convert the target "4" to "1"

In [15]:
from threading import Condition
twitter_data.replace({'target':{'positive':1}}, inplace=True)

Condition_mapping = {
    'Negative' : 0,
    'Positive' : 1,
    'Neutral'  : 0.5,
    'Irrelevant':-1
}

In [16]:
print(twitter_data['target'])


0        Positive
1        Positive
2        Positive
3        Positive
4        Positive
           ...   
46425     Neutral
46426     Neutral
46427     Neutral
46428     Neutral
46429     Neutral
Name: target, Length: 46430, dtype: object


In [17]:
 # checking the distribution of target column
twitter_data['target'].value_counts()


target
Positive      13728
Negative      12378
Neutral       11516
Irrelevant     8808
Name: count, dtype: int64

**0--------> Negative tweet
1--------> positive tweet
Stemming
steming is the process of reduceing a word to its roots word, example -actor, actress, acting = act**

In [18]:
#from nltk.stem import PorterStemmer
#import re
#twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)# 50 minutes to complete thus execution


In [19]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def stemming(text):
    """
    Stems text data while preserving efficiency and accuracy.

    Args:
        text (str): The text to be stemmed.

    Returns:
        str: The stemmed text.
    """

    # Download NLTK resources if not already present
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)

    # Tokenize text into lowercase words using NLTK word_tokenize
    words = word_tokenize(text.lower())

    # Remove stopwords and perform stemming
    stemmed_words = [PorterStemmer().stem(word) for word in words if word not in stopwords.words('english')]

    # Join stemmed words back into a string
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

# Assuming you have a DataFrame called 'twitter_data' with a column named 'text'
import pandas as pd
twitter_data['text'] = twitter_data['text'].astype(str)
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)



In [20]:
twitter_data.head()

,id,user platform,target,text,stemmed_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,"im get borderland murder ,"
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,"come border kill ,"
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,"im get borderland kill ,"
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,"im come borderland murder ,"
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,"im get borderland 2 murder ,"


In [21]:
print(twitter_data['stemmed_content'])

0                               im get borderland murder ,
1                                       come border kill ,
2                                 im get borderland kill ,
3                              im come borderland murder ,
4                             im get borderland 2 murder ,
                               ...                        
46425    5g cell tower danger  get fact ! radiationhea...
46426    5g mobil tower danger - get fact ! radiationhe...
46427    5g mobil oper danger - argument fact ! radiati...
46428    5g cell tower becom danger  get key fact ! ra...
46429                                          5g cell tow
Name: stemmed_content, Length: 46430, dtype: object


In [22]:
twitter_data['new_target'] = twitter_data['target'].map(Condition_mapping)
Condition_mapping = {
    'Negative' : 0,
    'Positive' : 1,
    'Neutral'  : 0.5,
    'Irrelevant':-1
}
train_X = twitter_data['stemmed_content'].values
train_Y = twitter_data['new_target'].values

In [23]:
print(train_X)

['im get borderland murder ,' 'come border kill ,'
 'im get borderland kill ,' ...
 '5g mobil oper danger - argument fact ! radiationhealthrisks.com / 5g-cell-tow ...'
 '5g cell tower becom danger \x96 get key fact ! radiationhealthrisks.com/5g-cell-tow'
 '5g cell tow']


In [24]:
Y_train = 'Y'
Y_train = np.array(train_Y)

In [25]:
print(Y_train)

[1.  1.  1.  ... 0.5 0.5 0.5]


# Spliting the data into training data and test data

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler  # For undersampling (optional)

# Assuming 'twitter_data' is a pandas DataFrame

try:
    # Separate features (X) and target variable (y)
    X = twitter_data['stemmed_content'].values.reshape(-1, 1)  # Reshape for compatibility
    y = twitter_data['target'].values

    # Check for class imbalance
    class_counts = pd.Series(y).value_counts()
    min_class_size = class_counts.min()

    if min_class_size < 2:
        print("Warning: Class imbalance detected. The least populated class has only",
              min_class_size, "member(s).")

        # Option 1: Undersample majority class (if applicable)
        if min_class_size > 1:  # Undersample only if there are more than 1 member
            majority_class = class_counts.idxmax()
            minority_class = class_counts.idxmin()

            # Undersample using RandomUnderSampler (optional)
            rus = RandomUnderSampler(sampling_strategy=minority_class)
            X_train, y_train = rus.fit_resample(X, y)
            X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=2)

        # Option 2: Oversample minority class (consider data augmentation)
        else:
            print("Option 2: Oversampling minority class is recommended, but not implemented here.")
            # Implement oversampling techniques (e.g., SMOTE) if applicable

    else:
        # Split data with stratification (if class balance is okay)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

except KeyError as e:
    print(f"Error: Columns '{e.name}' not found in 'twitter_data'. Please check your DataFrame.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

else:
    print("Data successfully split into training and test sets!")


Data successfully split into training and test sets!


In [27]:
print(X.shape, X_train.shape,X_test.shape)

(46430, 1) (37144, 1) (9286, 1)


In [28]:
print (X_train)

[['revlon brush realli work ! littl ladi go cop take shit rn amazon ! \x92 ? never need go pay get blow ever !']
 ['? 4 ? ? ? ? ! ! ! ! borderland 3 type night ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !']
 ['bit late final got master tier best hero redd.it/j4otzm']
 ...
 ['lahor high court ban pubg pakistan neg impact children .. correct decis ?']
 ['march great month : . . . 3 new streamer . . 4th place cod finish . . first csgo adv playoff . . april : . . new sponsor tonight 7']
 ['< unk > servic - major pharmaceut compani includ h , @ comcast , @ verizon say may termin servic chang charg individu impact coronaviru ... hudsonvalley.news12.com/story/42065161 ...']]


In [58]:
print (Y_train)


[1.  1.  1.  ... 0.5 0.5 0.5]


# # training the machine learning model

**logestic regression**

In [53]:
import pandas as pd  # Assuming your text data is in a pandas DataFrame
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert lists to Pandas Series
X_train = pd.Series(X_train)
X_test = pd.Series(X_test)

# Convert data type to string
X_train = X_train.astype(str).tolist()
X_test = X_test.astype(str).tolist()

vectorizer = TfidfVectorizer()

def _preprocess(doc, accent_function, lower):
    """
    Preprocess the given document.
    """
    if lower:
        doc = str(doc).lower()  # Convert to string and then lowercase
    if accent_function is not None:
        doc = accent_function(doc)
    return doc

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [55]:
print (X_train)

  (0, 5660)	0.18067720612827398
  (0, 2725)	0.29574395220956906
  (0, 6811)	0.13518258458719384
  (0, 11541)	0.21153193768732229
  (0, 10661)	0.18036946811274704
  (0, 10700)	0.18279806442304594
  (0, 17340)	0.1492240509275277
  (0, 1564)	0.15449531487065007
  (0, 13095)	0.25912742254463145
  (0, 13877)	0.15943328873870624
  (0, 15094)	0.18677460561067297
  (0, 4015)	0.2793952286135677
  (0, 6969)	0.2880259390647739
  (0, 9084)	0.2578189423148563
  (0, 9396)	0.21312882365510727
  (0, 17167)	0.17065463281782797
  (0, 12642)	0.15285149585200256
  (0, 3004)	0.3487388894124962
  (0, 12960)	0.3609916768009546
  (1, 10774)	0.5598662331150803
  (1, 16015)	0.7018517289918956
  (1, 2839)	0.4404020339742054
  (2, 8429)	0.45344713983652224
  (2, 8372)	0.2365770375052643
  (2, 12699)	0.33216428911354
  :	:
  (37142, 4245)	0.17158111358904593
  (37142, 6142)	0.16622095621215674
  (37142, 7131)	0.1590247341353081
  (37142, 10702)	0.292904478470335
  (37143, 701)	0.2821566567271654
  (37143, 10714)	0

In [56]:
print (X_test)

  (0, 10887)	0.529098927846018
  (0, 10795)	0.6724199133000575
  (0, 3858)	0.5175961599062897
  (1, 5165)	1.0
  (2, 17340)	0.18841235463106284
  (2, 16561)	0.21981858026987314
  (2, 15569)	0.18893625496032881
  (2, 14947)	0.268783400551709
  (2, 12819)	0.29502839681752047
  (2, 12224)	0.40231298989496234
  (2, 11817)	0.156748473330747
  (2, 9396)	0.26909940626199447
  (2, 7742)	0.2973135204759234
  (2, 7370)	0.23107993347664635
  (2, 6635)	0.14493704759693057
  (2, 6533)	0.22041961693116824
  (2, 5938)	0.2548917808310289
  (2, 5660)	0.22812554426178114
  (2, 2591)	0.29971818084669394
  (2, 2311)	0.21007186440208403
  (3, 13877)	0.33668874797412446
  (3, 11817)	0.2621696208560445
  (3, 7712)	0.49160763110153444
  (3, 7520)	0.6728884931699979
  (3, 2311)	0.35135564559284055
  :	:
  (9282, 278)	0.27040423609561015
  (9282, 142)	0.28253561378198894
  (9283, 15685)	0.4950714198343982
  (9283, 15181)	0.43815434317404733
  (9283, 7133)	0.5896739433728356
  (9283, 5034)	0.4639069964206207
  (9

# training the machine learning model
logestic regression

In [68]:
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression
from sklearn.model_selection import train_test_split

model = LogisticRegression(max_iter=1000)  # Set max_iter to a valid integer value

#Reshape Y_train to have the same number of samples as X_train
Y_train = Y_train.astype(int).reshape(-1, 1)   # Fix the shape of Y_train
#from sklearn.linear_model import LogisticRegression  # Import LogisticRegression

#model = LogisticRegression(max_iter=1000)  # Create a LogisticRegression model
# Reshape Y_train to have the same number of samples as X_train
Y_train = Y_train[:37144]  # Truncate Y_train to match the length of X_train

In [69]:
model.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

# Model evaluation

Accuracy score

In [71]:
from sklearn.metrics import accuracy_score
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)


print ('accuracy score on the training data: ', training_data_accuracy)

accuracy score on the training data:  0.5943355589058799


In [74]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test, X_test_prediction)


print ('accuracy score on the training data: ', test_data_accuracy)

accuracy score on the training data:  0.0


# using the saved model for future prediction

In [76]:
import pickle

In [77]:
filename = 'trained_model1.sav'
pickle.dump(model, open(filename,'wb'))

In [80]:
# loading the savedmodel
loaded_model = pickle.load(open('/content/trained_model1.sav','rb'))